In [1]:
from keras.applications.resnet50 import ResNet50
from quiver_engine import server
from keras.models import Model

Using TensorFlow backend.


# Visualizing CNN's intermediate layer output

We'll use [quiver](https://github.com/keplr-io/quiver) which works for Keras models.

![panda](../../assets/panda.png)

<center><i>outputs of <b>Resnet50</b>'s max_pooling2d_1 layer</i></center>

## Model setup

We use InceptionV3 but any model from `keras.application` could be used.

In [2]:
input_shape = (224, 224, 3) # size of our image: width x height x number of channels

model = ResNet50(
  include_top=False,
  weights='imagenet',
  input_shape=input_shape # quiver is going to need that
)

Select intermediate layer (so that the graph is not too big)

In [3]:
len(model.layers_by_depth)

166

In [4]:
intermediate_layer = model.get_layer(name="add_3")

truncated_model = Model(inputs=[model.input], outputs=[intermediate_layer.output])

In [5]:
print("Number of layers in truncated model:", len(truncated_model.layers))
truncated_model.summary()

Number of layers in truncated model: 36
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
_____________________________________________________________________

Sanity check - this should be `(None, 224, 224, 3)` (`None` corresponds to batch size) 

In [ ]:
truncated_model.get_input_shape_at(0)

(None, 224, 224, 3)

Be sure you ran 

```
make load_101_categories
```

before you run that to setup the images.

Also you could point *quiver* to other directory with images.

Let's run the visualization engine!

In [ ]:
imgs_path = "../../data/101_ObjectCategories/panda"

server.launch(
  truncated_model,
  temp_folder="../../data/tmp",
  input_folder=imgs_path,
  port=5000
)

Starting webserver from: /opt/anaconda3/envs/nnets/lib/python3.5/site-packages/quiver_engine


::ffff:127.0.0.1 - - [2018-02-04 12:11:52] "GET /model HTTP/1.1" 200 32476 0.015062
::ffff:127.0.0.1 - - [2018-02-04 12:11:52] "GET /inputs HTTP/1.1" 200 975 0.001692
